# Machine Learning project
By Mattia Barbiere

In [138]:
import sys
import sklearn
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import warnings


In [139]:
df = pd.read_csv("mldata_0003151393.csv", index_col=0)

In [140]:
for i in df.columns:
    print(df[i].mean())

2.0141666666666667
0.10157465347525149
-0.16671013516841957
0.13298123610853088
-0.08711174904388803
0.1255218199715846
0.253445671295189
0.31311224920664077
0.10700662739801253
0.4883202091121518
0.09352928843167466
0.08336231126198083
-0.4328550967032481
0.08003992543083605
-0.6011694724154556
0.044112453245636896
0.4762917149390366
0.0964802548840801
0.09264297832785592
0.08221226958318986
-0.45667411146032216


In [141]:
for i in df.columns:
    print(df[i].std())

1.4173826054024028
2.0782991205770545
1.981570830889979
0.967589995397125
2.0536833525361318
1.9387203823072805
2.006632337119499
3.2000073091607253
1.0100857402951804
2.187283553031497
1.012772077976055
1.0522024976350732
3.9191048732283083
0.9924997998391863
3.672404130054445
1.0090592977395891
1.9158544349658608
0.9897959521219011
1.005401262922337
1.9611801382505776
1.9067285108827712


In [142]:
df.head(10)

,label,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20
0,1,-2.887351,2.368955,-0.625925,-3.978334,-0.326955,4.055608,-0.235264,0.273827,-0.517484,...,1.510891,-0.111374,0.940897,-3.955810,-0.363500,0.199346,0.461904,-0.126972,1.626261,-0.182263
1,3,1.334865,-1.751757,-0.578244,-2.852914,-0.046243,-0.404612,4.838986,2.054794,0.641435,...,-1.172510,-0.015095,0.175453,-2.446950,-0.718050,0.596897,0.170454,-1.004904,2.982466,-3.522866
2,0,-1.073810,3.398483,-1.099216,1.529714,1.319205,4.183233,-4.627822,0.452511,3.557216,...,0.821012,6.678114,-0.540001,1.582700,0.466075,-2.284209,0.133034,0.216640,2.448328,2.277177
3,3,-1.459105,0.604342,-0.052011,0.196076,2.174851,-0.732217,-1.997095,2.443736,3.655130,...,0.491459,-0.112123,1.388338,-2.690753,0.997441,-0.957406,-1.423567,-0.276991,0.858894,-0.212745
4,2,0.508266,3.532480,-0.196915,-3.370030,0.177465,0.025390,2.840482,1.251280,-3.536680,...,0.758948,-5.192279,-0.341884,-2.078780,1.978526,1.532484,2.001019,-1.013421,-1.295467,-1.002012
5,4,-0.316656,-0.922855,1.447635,1.699483,0.022101,0.473463,-0.126300,0.966990,1.545700,...,-0.574484,2.309736,-1.206329,0.574633,-0.158552,-1.874306,0.576478,-0.918869,-0.961063,0.828915
6,3,-0.350357,-0.280938,1.572685,0.922978,2.159589,-1.828143,1.953110,1.780278,0.755949,...,-0.297084,1.999719,1.578167,-0.109991,0.199291,-2.984111,0.158336,-1.480244,-0.604203,-1.328385
7,2,1.512664,2.661731,-1.997446,-0.114986,-0.558272,-0.272194,2.245315,2.066404,1.031266,...,-1.191092,-4.413065,1.049424,-2.668520,-0.566365,0.140097,1.048035,-0.162599,-2.040033,-0.893740
8,2,-1.331660,0.894030,0.167612,-2.500779,2.216463,5.035682,-5.196242,0.905760,2.859414,...,-0.008510,4.528739,0.215946,0.495931,-1.454981,0.590947,-0.027441,0.839361,2.616551,3.506210
9,3,0.437816,-0.601652,-0.031859,-1.010379,-1.492245,1.222452,3.299241,0.504061,-0.439967,...,-0.762156,-3.027410,-1.525858,-2.826487,1.820201,1.583277,-0.455038,-1.004994,-0.963731,-2.737519


# Standerdizing the model

In [143]:
df.isna().sum()

label         0
feature_1     0
feature_2     0
feature_3     0
feature_4     0
feature_5     0
feature_6     0
feature_7     0
feature_8     0
feature_9     0
feature_10    0
feature_11    0
feature_12    0
feature_13    0
feature_14    0
feature_15    0
feature_16    0
feature_17    0
feature_18    0
feature_19    0
feature_20    0
dtype: int64

In [144]:
df['label'].value_counts(sort=False)

1    223
3    246
0    244
2    246
4    241
Name: label, dtype: int64

In [145]:
from sklearn.model_selection import train_test_split

#Train test split
X_train, X_test, y_train, y_test = train_test_split(df[df.columns[1:]], df['label'], test_size=0.2, random_state=42)

In [146]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Making the one pipeline for each model
pipe_logi = Pipeline([
        ('std_scaler', None),
        ('logi_model', LogisticRegression(random_state=0))
    ])

pipe_knn = Pipeline([
        ('std_scaler', None),
        ('KNN_model', KNeighborsClassifier())
    ])

#Fitting
pipe_logi.fit(X_train,y_train)
pipe_knn.fit(X_train,y_train)

print(pipe_logi.predict(X_test))
print(pipe_knn.predict(X_test))

print("Logistic Regression initial score: ", pipe_logi.score(X_test, y_test))
print("K Nearest Neighbor initial score: ", pipe_knn.score(X_test, y_test))

[2 2 2 0 4 0 4 0 4 1 0 4 0 2 4 3 3 3 1 3 4 4 4 0 2 0 3 4 4 3 0 2 0 4 3 0 4
 0 4 4 0 3 2 0 4 0 2 2 2 3 1 1 4 4 3 2 4 0 4 2 3 1 3 2 2 1 0 3 3 1 0 2 4 0
 3 3 4 0 0 0 0 2 4 3 3 3 0 2 0 2 2 2 2 0 2 2 0 3 4 0 2 4 2 4 4 4 1 4 4 2 4
 0 4 2 3 4 1 2 2 4 4 2 4 1 4 0 2 2 2 0 4 3 2 2 2 4 4 3 4 3 2 1 3 2 2 3 3 4
 0 1 3 4 3 0 0 4 0 4 3 4 4 4 4 4 4 0 4 1 2 2 3 2 4 0 4 3 4 3 0 4 2 3 2 3 3
 3 0 4 2 1 2 4 2 0 2 3 2 2 0 4 4 3 3 0 0 3 4 3 4 2 1 0 3 4 3 0 3 3 2 2 3 4
 1 4 0 0 4 0 1 0 3 2 4 4 0 0 2 1 4 1]
[4 4 4 0 4 0 1 1 4 1 4 1 2 2 2 1 3 3 3 3 4 4 4 4 2 0 0 2 4 3 2 0 0 1 1 0 1
 2 2 0 4 4 2 4 4 1 0 4 4 0 1 2 4 0 3 3 2 0 2 3 0 1 3 2 2 1 0 0 4 1 1 2 4 0
 2 1 3 2 3 0 2 1 4 3 3 1 0 2 1 1 4 2 4 0 2 2 4 0 4 4 0 4 4 4 0 4 1 4 4 2 0
 3 4 2 3 3 4 2 1 4 4 2 0 1 2 1 3 0 3 1 3 1 2 2 1 4 4 4 4 1 0 3 0 2 2 3 3 3
 0 4 3 0 3 0 0 0 0 3 3 4 3 4 2 4 0 1 4 1 1 2 3 2 4 2 1 3 2 1 1 4 3 4 1 1 1
 4 2 4 2 1 2 4 1 0 3 1 2 1 0 0 3 0 3 0 0 1 1 3 3 0 0 0 0 4 3 3 1 3 2 2 3 4
 1 2 0 3 0 0 1 0 0 2 4 1 0 0 2 1 1 3]
Logistic Regression init

In [147]:
pipe_knn.get_params()

{'memory': None,
 'steps': [('std_scaler', None), ('KNN_model', KNeighborsClassifier())],
 'verbose': False,
 'std_scaler': None,
 'KNN_model': KNeighborsClassifier(),
 'KNN_model__algorithm': 'auto',
 'KNN_model__leaf_size': 30,
 'KNN_model__metric': 'minkowski',
 'KNN_model__metric_params': None,
 'KNN_model__n_jobs': None,
 'KNN_model__n_neighbors': 5,
 'KNN_model__p': 2,
 'KNN_model__weights': 'uniform'}

In [148]:
pipe_logi.get_params()

{'memory': None,
 'steps': [('std_scaler', None),
  ('logi_model', LogisticRegression(random_state=0))],
 'verbose': False,
 'std_scaler': None,
 'logi_model': LogisticRegression(random_state=0),
 'logi_model__C': 1.0,
 'logi_model__class_weight': None,
 'logi_model__dual': False,
 'logi_model__fit_intercept': True,
 'logi_model__intercept_scaling': 1,
 'logi_model__l1_ratio': None,
 'logi_model__max_iter': 100,
 'logi_model__multi_class': 'auto',
 'logi_model__n_jobs': None,
 'logi_model__penalty': 'l2',
 'logi_model__random_state': 0,
 'logi_model__solver': 'lbfgs',
 'logi_model__tol': 0.0001,
 'logi_model__verbose': 0,
 'logi_model__warm_start': False}

In [149]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
warnings.simplefilter('ignore')

#Grid search for K Nearest Neighbors
g_param_knn = {'KNN_model__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                'KNN_model__leaf_size': [10, 20, 30, 40, 50],
                'KNN_model__n_neighbors': [2, 3, 4, 5, 6, 7],
                'KNN_model__p': [0, 1, 2],
                'KNN_model__weights': ['uniform', 'distance']}
g_knn = GridSearchCV(pipe_knn, g_param_knn, cv=KFold(shuffle=True, n_splits=5), scoring='accuracy', n_jobs=-1)
g_knn.fit(X_train, y_train)

#Results of Grid Search
print(g_knn.best_score_)
g_knn.best_params_



0.7052083333333333


{'KNN_model__algorithm': 'auto',
 'KNN_model__leaf_size': 10,
 'KNN_model__n_neighbors': 6,
 'KNN_model__p': 2,
 'KNN_model__weights': 'uniform'}

In [150]:
from sklearn.model_selection import cross_val_score

pipe_knn.set_params(**g_knn.best_params_)
pipe_knn.fit(X_train,y_train)
print("K Nearest Neighbor score after Grid Search: ", pipe_knn.score(X_test, y_test))
print("K Nearest Neighbor mean CV score after Grid Search: ", cross_val_score(pipe_knn, X_train, y_train, cv=KFold(shuffle=True, n_splits=5), scoring="accuracy").mean())


K Nearest Neighbor score after Grid Search:  0.7416666666666667
K Nearest Neighbor mean CV score after Grid Search:  0.6864583333333334


In [151]:
#Grid search for Logistic Regression
g_param_logi = {'std_scaler': [StandardScaler(), None],
                'logi_model__penalty' : ['l1', 'l2', 'elasticnet', None],
               'logi_model__C' : np.logspace(-5, 5, 20),
               'logi_model__multi_class': ['multinomial', 'ovr'],
               'logi_model__l1_ratio': [0, 0.25, 0.5, 0.75, 1, None],
               'logi_model__class_weight': [None, 'balanced']
               }
g_logi = GridSearchCV(pipe_logi, g_param_logi, cv=KFold(shuffle=True, n_splits=5), scoring='accuracy', n_jobs=-1)
g_logi.fit(X_train, y_train)

#Results of Grid Search
print(g_logi.best_score_)
g_logi.best_params_


0.4729166666666667


{'logi_model__C': 0.004281332398719391,
 'logi_model__class_weight': None,
 'logi_model__l1_ratio': 0,
 'logi_model__multi_class': 'ovr',
 'logi_model__penalty': 'l2',
 'std_scaler': None}

In [152]:
pipe_logi.set_params(**g_logi.best_params_)
pipe_logi.fit(X_train,y_train)
print("Logistic Regression score after Grid Search: ", pipe_logi.score(X_test, y_test))
print("Logistic Regression mean CV score after Grid Search: ", cross_val_score(pipe_logi, X_train, y_train, cv=KFold(shuffle=True, n_splits=5), scoring="accuracy").mean())


Logistic Regression score after Grid Search:  0.45416666666666666
Logistic Regression mean CV score after Grid Search:  0.471875
